M5 Forecasting - Accuracy

Note: This is one of the two complementary competitions that together comprise the M5 forecasting challenge. Can you estimate, as precisely as possible, the point forecasts of the unit sales of various products sold in the USA by Walmart?

How much camping gear will one store sell each month in a year? To the uninitiated, calculating sales at this level may seem as difficult as predicting the weather. Both types of forecasting rely on science and historical data. While a wrong weather forecast may result in you carrying around an umbrella on a sunny day, inaccurate business forecasts could result in actual or opportunity losses. In this competition, in addition to traditional forecasting methods you’re also challenged to use machine learning to improve forecast accuracy.

The Makridakis Open Forecasting Center (MOFC) at the University of Nicosia conducts cutting-edge forecasting research and provides business forecast training. It helps companies achieve accurate predictions, estimate the levels of uncertainty, avoiding costly mistakes, and apply best forecasting practices. The MOFC is well known for its Makridakis Competitions, the first of which ran in the 1980s.

In this competition, the fifth iteration, you will use hierarchical sales data from Walmart, the world’s largest company by revenue, to forecast daily sales for the next 28 days. The data, covers stores in three US States (California, Texas, and Wisconsin) and includes item level, department, product categories, and store details. In addition, it has explanatory variables such as price, promotions, day of the week, and special events. Together, this robust dataset can be used to improve forecasting accuracy.

If successful, your work will continue to advance the theory and practice of forecasting. The methods used can be applied in various business areas, such as setting up appropriate inventory or service levels. Through its business support and training, the MOFC will help distribute the tools and knowledge so others can achieve more accurate and better calibrated forecasts, reduce waste and be able to appreciate uncertainty and its risk implications.

Evaluation:
This competition uses a Weighted Root Mean Squared Scaled Error (RMSSE). Extensive details about the metric, scaling, and weighting can be found in the [M5 Participants Guide](https://mofc.unic.ac.cy/m5-competition/).

It is important to notice that this notebook must be seen together with the notebook "M5 Forecasting - Accuracy Data study.ipynb".
All the necessary data verification and the calculation of the parameters of the ARIMA(p, i, q) is done in "M5 Forecasting - Accuracy Data study.ipynb". 
This notebook have only the constructions of new variables.

In [1]:
import gc
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from statsmodels.tsa.arima_model import ARIMA

pd.options.display.max_columns = 92

In [2]:
'''
Function to reduce memory usage.
From: https://www.kaggle.com/ragnar123/very-fst-model
'''
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
#Creates dummies
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name], prefix = column_name)
    df = pd.concat([df, dummies], axis = 1)
    return df

In [4]:
#Load data.
calendar = pd.read_csv('calendar.csv')
calendar = reduce_mem_usage(calendar)
sell_prices = pd.read_csv('sell_prices.csv')
sell_prices = reduce_mem_usage(sell_prices)
sales_train_validation = pd.read_csv('sales_train_validation.csv')
sales_train_validation = reduce_mem_usage(sales_train_validation)

Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Mem. usage decreased to 95.00 Mb (78.7% reduction)


File 1: “calendar.csv”

Contains information about the dates the products are sold.

     date: The date in a “y-m-d” format.

     wm_yr_wk: The id of the week the date belongs to.
    
     weekday: The type of the day (Saturday, Sunday, …, Friday).
    
     wday: The id of the weekday, starting from Saturday.
    
     month: The month of the date.
    
     year: The year of the date.
    
     event_name_1: If the date includes an event, the name of this event.
    
     event_type_1: If the date includes an event, the type of this event.
    
     event_name_2: If the date includes a second event, the name of this event.
    
     event_type_2: If the date includes a second event, the type of this event.
    
     snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA, TX or WI allow SNAP2 purchases on the examined date. 1 indicates that SNAP purchases are allowed.

File 2: “sell_prices.csv”

Contains information about the price of the products sold per store and date.

     store_id: The id of the store where the product is sold.

     item_id: The id of the product.

     wm_yr_wk: The id of the week.

     sell_price: The price of the product for the given week/store. The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. Note that although prices are constant at weekly basis, they may change through time (both training and test set).
    
    
Considering that we have only 6,841,121 prices, there are  1,757,059 (8,598,180 - 6,841,121) products that were not sold in a given week.

File 3: “sales_train.csv”

Contains the historical daily unit sales data per product and store.

     item_id: The id of the product.

     dept_id: The id of the department the product belongs to.

     cat_id: The id of the category the product belongs to.

     store_id: The id of the store where the product is sold.

     state_id: The State where the store is located.

     d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29.
    
    
We have 282 and 30,490 differents items. Therefore, the ideal would be to have 8,598,180 (282 x 30,400) prices. 

In [5]:
#Variables based on calendar
#Seasons
calendar['date'] = pd.to_datetime(calendar['date'])
calendar['day'] = calendar['date'].dt.day

calendar.loc[np.in1d(calendar['month'], [3, 4, 5]), 'season'] = 1 #'spring'
calendar.loc[np.in1d(calendar['month'], [6, 7, 8]), 'season'] = 2 #'summer'
calendar.loc[np.in1d(calendar['month'], [9, 10, 11]), 'season'] = 3 #'fall'
calendar.loc[np.in1d(calendar['month'], [12, 1, 2]), 'season'] = 4 #'winter'
calendar['season'] = calendar['season'].astype('int8')

calendar.loc[np.in1d(calendar['day'], [1, 2, 3, 4, 5, 6, 7]), 'month_fase'] = 1# 'start'
calendar.loc[np.in1d(calendar['day'], [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]), 'month_fase'] = 2 #'middle'
calendar.loc[np.in1d(calendar['day'], [22, 23, 24, 25, 26, 27, 28, 29, 30, 31]), 'month_fase'] = 3 #'end'
calendar['month_fase'] = calendar['month_fase'].astype('int8')

calendar.loc[np.in1d(calendar['wday'], [1, 2]), 'week_fase'] = 1 #'weekend'
calendar.loc[np.in1d(calendar['wday'], [3, 4, 5]), 'week_fase'] = 2 #'start'
calendar.loc[np.in1d(calendar['wday'], [6, 7]), 'week_fase'] = 3 #'end'
calendar['week_fase'] = calendar['week_fase'].astype('int8')

calendar.drop(['date', 'day', 'weekday', 'year'], inplace = True, axis = 1)

#Fix variables event_name and event_type
#Make new holiday variable
calendar['event_name_1_t3'] = calendar['event_name_1'].shift(-3)
calendar['event_name_2_t3'] = calendar['event_name_2'].shift(-3)
calendar['event_name_1_t7'] = calendar['event_name_1'].shift(-7)
calendar['event_name_2_t7'] = calendar['event_name_2'].shift(-7)

calendar['event_type_1_t3'] = calendar['event_type_1'].shift(-3)
calendar['event_type_2_t3'] = calendar['event_type_2'].shift(-3)
calendar['event_type_1_t7'] = calendar['event_type_1'].shift(-7)
calendar['event_type_2_t7'] = calendar['event_type_2'].shift(-7)

#Categorize holidays
def cat_holidays(holiday):
    calendar[holiday] = np.logical_or(np.in1d(calendar['event_name_1'], holiday), np.in1d(calendar['event_name_1'], holiday))
    
    calendar[holiday + '_t3'] = np.logical_or(np.in1d(calendar['event_name_1_t3'], holiday)
                                                               , np.in1d(calendar['event_name_2_t3'], holiday))
    
    calendar[holiday + '_t7'] = np.logical_or(np.in1d(calendar['event_name_1_t7'], holiday)
                                                               , np.in1d(calendar['event_name_2_t7'], holiday))
    
    calendar[holiday + '_near'] = 0
    calendar[holiday + '_near'] = calendar[holiday + '_t3'].rolling(4).sum()
    calendar[holiday + '_near'].fillna(0, inplace = True)
    
    calendar[holiday + '_week'] = 0
    calendar[holiday + '_week'] = calendar[holiday + '_t7'].rolling(8).sum()
    calendar[holiday + '_week'].fillna(0, inplace = True)
    calendar[holiday + '_weekend']  = np.logical_and(calendar[holiday + '_week'], np.in1d(calendar['week_fase'], 1))
    
    calendar[holiday + '_near'] = calendar[holiday + '_near'].astype('bool')
    calendar[holiday + '_week'] = calendar[holiday + '_week'].astype('bool')
    
    calendar.drop([holiday + '_t3', holiday + '_t7'], inplace = True, axis = 1)
        
        
def cat_holiday_type(holiday_type):
    calendar[holiday_type] = np.logical_or(np.in1d(calendar['event_type_1'], holiday_type), np.in1d(calendar['event_type_2'], holiday_type))
      
    calendar[holiday_type + '_t3'] = np.logical_or(np.in1d(calendar['event_type_1_t3'], holiday_type)
                                                               , np.in1d(calendar['event_type_2_t3'], holiday_type))
    
    calendar[holiday_type + '_t7'] = np.logical_or(np.in1d(calendar['event_type_1_t7'], holiday_type)
                                                               , np.in1d(calendar['event_type_2_t7'], holiday_type))
    
    calendar[holiday_type + '_near'] = 0
    calendar[holiday_type + '_near'] = calendar[holiday_type + '_t3'].rolling(4).sum()
    calendar[holiday_type + '_near'].fillna(0, inplace = True)
    
    calendar[holiday_type + '_week'] = 0
    calendar[holiday_type + '_week'] = calendar[holiday_type + '_t7'].rolling(8).sum()
    calendar[holiday_type + '_week'].fillna(0, inplace = True)
    calendar[holiday_type + '_weekend']  = np.logical_and(calendar[holiday_type + '_week'], np.in1d(calendar['week_fase'], 1))
      
    calendar[holiday_type + '_near'] = calendar[holiday_type + '_near'].astype('bool')
    calendar[holiday_type + '_week'] = calendar[holiday_type + '_week'].astype('bool')
        
    calendar.drop([holiday_type + '_t3', holiday_type + '_t7'], inplace = True, axis = 1)

    
holidays = calendar['event_name_1'].unique()[1:]
holiday_type = calendar['event_type_1'].unique()[1:]

for e in holidays:
    cat_holidays(e)
    
for e in holiday_type:
    cat_holiday_type(e)
    
    
calendar.drop(['event_name_1', 'event_name_2', 'event_type_1', 'event_type_2'
               , 'event_name_1_t3', 'event_name_1_t7'
               , 'event_name_2_t3', 'event_name_2_t7'
               , 'event_type_1_t3', 'event_type_1_t7'
               , 'event_type_2_t3', 'event_type_2_t7'], inplace = True, axis = 1)

calendar = reduce_mem_usage(calendar)  #Mem. usage decreased to  0.29 Mb (10.3% reduction)

Mem. usage decreased to  0.29 Mb (0.0% reduction)


In [6]:
#Transpose sales_train_validation so the days goes to rows.
sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')

In [7]:
#Define 2 holdout
holdout1 = pd.read_csv('sample_submission.csv')
holdout1 = reduce_mem_usage(holdout1)
holdout2 = pd.read_csv('sample_submission.csv')
holdout2 = reduce_mem_usage(holdout2)


#One holdout is related to forecast between day 1914 and 1941.
#The second, between day 1942 and 1969
test1 = holdout1.copy()
test2 = holdout2.copy()
test1.columns = ['id', 'd_1914', 'd_1915', 'd_1916', 'd_1917', 'd_1918', 'd_1919', 'd_1920', 'd_1921', 'd_1922', 'd_1923',
                    'd_1924', 'd_1925', 'd_1926', 'd_1927', 'd_1928', 'd_1929', 'd_1930', 'd_1931', 
                    'd_1932', 'd_1933', 'd_1934', 'd_1935', 'd_1936', 'd_1937', 'd_1938', 'd_1939', 'd_1940', 'd_1941']
test2.columns = ['id', 'd_1942', 'd_1943', 'd_1944', 'd_1945', 'd_1946', 'd_1947', 'd_1948', 'd_1949', 'd_1950', 'd_1951', 
                    'd_1952', 'd_1953', 'd_1954', 'd_1955', 'd_1956', 'd_1957', 'd_1958', 'd_1959', 
                    'd_1960', 'd_1961', 'd_1962', 'd_1963', 'd_1964', 'd_1965', 'd_1966', 'd_1967', 'd_1968', 'd_1969']

Mem. usage decreased to  2.09 Mb (84.5% reduction)
Mem. usage decreased to  2.09 Mb (84.5% reduction)


In [8]:
#Concat our train database with the holdouts. 

#Get information about the product id.
product_id = sales_train_validation[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']].drop_duplicates()

# merge with product table
validation_bool = test1['id'].str.contains("_validation") 
test1 = test1[validation_bool]
test1 = test1.merge(product_id, how = 'left', on = 'id')
test2 = test2[validation_bool]
test2 = test2.merge(product_id, how = 'left', on = 'id')

#Transpose so the days goes to rows.
test1 = pd.melt(test1, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
test2 = pd.melt(test2, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')

#Define what is train and what is test.
sales_train_validation['part'] = 'train'
test1['part'] = 'test1'
test2['part'] = 'test2'   #Test2 will be used just after june 1. Before that, we can use it after forecastinf test 1.

In [9]:
#Concat sales_train_validation and test1. test2 will be concated after we develop some demand variables
data = pd.concat([sales_train_validation, test1], axis = 0) 
del sales_train_validation, test1

In [10]:
#make new demand related variables
#It is necessary a shift of 28 days so these variables can be calculated for the forecast.
#Considering the ARIMA model we will not use variables based on the lag or mean of thedemand   
# data['demand_lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
# data['demand_rolling_mean_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
# data['demand_rolling_mean_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
# data['demand_rolling_mean_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
# data['demand_rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
# data['demand_rolling_mean_t365'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(365).mean())
data['demand_rolling_std_t7'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
data['demand_rolling_std_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
data['demand_rolling_std_t90'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).std())
data['demand_rolling_std_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).std())
data['demand_rolling_std_t365'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(365).std())
data['demand_rolling_skew_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).skew())
data['demand_rolling_kurt_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).kurt())

data = reduce_mem_usage(data)

Mem. usage decreased to 4966.67 Mb (32.3% reduction)


In [11]:
#Concat data and test2
data = pd.concat([data, test2], axis = 0, sort = True)
del test2

#Merge with calendar.
data = pd.merge(data, calendar, how = 'left', left_on = ['day'], right_on = ['d'])
del calendar
gc.collect()
data.drop(['d'], inplace = True, axis = 1)

In [12]:
#Merge with sell_prices.
data = pd.merge(data, sell_prices, how = 'left', on = ['store_id', 'item_id', 'wm_yr_wk'])

#Delete rows with products that had not have yet your first sell.
data = data.dropna(subset=['sell_price']) 

In [13]:
#make new price related variables
def create_bins_price_variation(df, var_name_in):
    cut_points = [-999, -0.25, -0.10, -0.05, -0.001, 0.05, 0.10, 0.25, 999]
    label_names = ["--0.25", "-0.25-0.10", "-0.10-0.5", "-0.05-0", "0-0.05", "0.05-0.10", "0.10-0.25", "0.25++"]
    df[var_name_in + '_bin'] = pd.cut(df[var_name_in], cut_points, labels = label_names)
    #create dummy
    df = create_dummies(df, var_name_in + '_bin')
    gc.collect()  
    df.drop([var_name_in, var_name_in + '_bin', var_name_in + '_bin_0-0.05'], inplace = True, axis = 1)  # var_name_in + '_bin_0-0.05' is dropped to avoid correlatin with the others bins.
    return df


data['lag_price_t1'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
data['price_change_t1'] = (data['sell_price'] - data['lag_price_t1']) / (data['lag_price_t1'])
data.drop(['lag_price_t1'], inplace = True, axis = 1)

#Create bins for price change
data = create_bins_price_variation(data, 'price_change_t1')


#t4 = four weeks, one month;
#t13 = 13 weeks, three months;
#t26 = 26 weeks, six month;
#t52 = 52 weeks, one year.
data['rolling_price_max_t4'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(4).max())
data['rolling_price_max_t13'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(13).max())
data['rolling_price_max_t26'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(26).max())
data['rolling_price_max_t52'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(52).max())

data['price_change_max_t4'] = (data['sell_price'] - data['rolling_price_max_t4']) / (data['rolling_price_max_t4'])
data['price_change_max_t13'] = (data['sell_price'] - data['rolling_price_max_t13']) / (data['rolling_price_max_t13'])
data['price_change_max_t26'] = (data['sell_price'] - data['rolling_price_max_t26']) / (data['rolling_price_max_t26'])
data['price_change_max_t52'] = (data['sell_price'] - data['rolling_price_max_t52']) / (data['rolling_price_max_t52'])

data.drop(['rolling_price_max_t4', 'rolling_price_max_t13', 'rolling_price_max_t26', 'rolling_price_max_t52'], inplace = True, axis = 1)

#Create bins for max price change
data = create_bins_price_variation(data, 'price_change_max_t4')
data = create_bins_price_variation(data, 'price_change_max_t13')
data = create_bins_price_variation(data, 'price_change_max_t26')
data = create_bins_price_variation(data, 'price_change_max_t52')


data['rolling_price_std_t4'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(4).std())
data['rolling_price_std_t13'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(13).std())
data['rolling_price_std_t26'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(26).std())
data['rolling_price_std_t52'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(52).std())

data['rolling_price_mean_t4'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(4).mean())
data['rolling_price_mean_t13'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(13).mean())
data['rolling_price_mean_t26'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(26).mean())
data['rolling_price_mean_t52'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(52).mean())
                                                                                                  
data['price_change_mean_t4'] = (data['sell_price'] - data['rolling_price_mean_t4']) / (data['rolling_price_mean_t4'])
data['price_change_mean_t13'] = (data['sell_price'] - data['rolling_price_mean_t13']) / (data['rolling_price_mean_t13'])
data['price_change_mean_t26'] = (data['sell_price'] - data['rolling_price_mean_t26']) / (data['rolling_price_mean_t26'])                                                                                                   
data['price_change_mean_t52'] = (data['sell_price'] - data['rolling_price_mean_t52']) / (data['rolling_price_mean_t52'])

data.drop(['rolling_price_mean_t4', 'rolling_price_mean_t13', 'rolling_price_mean_t26', 'rolling_price_mean_t52'], inplace = True, axis = 1)


gc.collect()
#Create bins for mean price change                                                                                                  
data = create_bins_price_variation(data, 'price_change_mean_t4')
data = create_bins_price_variation(data, 'price_change_mean_t13')
data = create_bins_price_variation(data, 'price_change_mean_t26')
data = create_bins_price_variation(data, 'price_change_mean_t52')

In [14]:
#Verify if there is a snap sales in other state.
gc.collect()

data['snap_other_CA'] = 0
data['snap_other_CA_only'] = 0
data.loc[(np.in1d(data['snap_TX'], 1)) | (np.in1d(data['snap_WI'], 1)), 'snap_other_CA'] = 1
data.loc[(np.in1d(data['snap_other_CA'], 1)) & (np.in1d(data['snap_CA'], 0)), 'snap_other_CA_only'] = 1

data['snap_other_TX'] = 0
data['snap_other_TX_only'] = 0
data.loc[(np.in1d(data['snap_CA'], 1)) | (np.in1d(data['snap_WI'], 1)), 'snap_other_TX'] = 1
data.loc[(np.in1d(data['snap_other_TX'], 1)) & (np.in1d(data['snap_TX'], 0)), 'snap_other_TX_only'] = 1

data['snap_other_WI'] = 0
data['snap_other_WI_only'] = 0
data.loc[(np.in1d(data['snap_CA'], 1)) | (np.in1d(data['snap_TX'], 1)), 'snap_other_WI'] = 1
data.loc[(np.in1d(data['snap_other_WI'], 1)) & (np.in1d(data['snap_WI'], 0)), 'snap_other_WI_only'] = 1

data = reduce_mem_usage(data)

#Set the right snap per product.
data.loc[np.in1d(data['state_id'], 'CA'), 'snap'] = data['snap_CA']
data.loc[np.in1d(data['state_id'], 'CA'), 'snap_other'] = data['snap_other_CA']
data.loc[np.in1d(data['state_id'], 'CA'), 'snap_only_other'] = data['snap_other_CA_only']

data.loc[np.in1d(data['state_id'], 'TX'), 'snap'] = data['snap_TX']
data.loc[np.in1d(data['state_id'], 'TX'), 'snap_other'] = data['snap_other_TX']
data.loc[np.in1d(data['state_id'], 'TX'), 'snap_only_other'] = data['snap_other_TX_only']

data.loc[np.in1d(data['state_id'], 'WI'), 'snap'] = data['snap_WI']
data.loc[np.in1d(data['state_id'], 'WI'), 'snap_other'] = data['snap_other_WI']
data.loc[np.in1d(data['state_id'], 'WI'), 'snap_only_other'] = data['snap_other_WI_only']

data = reduce_mem_usage(data)

data.drop(['snap_CA', 'snap_TX', 'snap_WI'
                , 'snap_other_CA', 'snap_other_TX', 'snap_other_WI'
                , 'snap_other_CA_only', 'snap_other_TX_only', 'snap_other_WI_only'], inplace = True, axis = 1)

#Create dummies
columns = ['season', 'month', 'wday', 'month_fase', 'week_fase']

for col in columns:
    data = create_dummies(data, col)    #Tem que dar drop em alguma categoria
    
gc.collect()
#Delete old variables and at least one bin from each categorical variable.
data.drop(['season', 'month', 'wday', 'month_fase', 'week_fase'
           , 'season_3', 'month_5', 'wday_5', 'month_fase_2', 'week_fase_2'], inplace = True, axis = 1)

#Considering that we will estimate a model by product, there are some more variables the we can drop.
data.drop(['cat_id', 'dept_id', 'item_id', 'state_id', 'store_id'], inplace = True, axis = 1)
data = reduce_mem_usage(data)

Mem. usage decreased to 14249.02 Mb (11.8% reduction)
Mem. usage decreased to 15802.16 Mb (4.9% reduction)
Mem. usage decreased to 14436.44 Mb (0.0% reduction)


In [15]:
#Add informatin for tje ARIMA(p, i, q)
df_piq_table = pd.read_csv('df_piq_table.csv')
data = pd.merge(data, df_piq_table, how = 'left', left_on = ['id'], right_on = ['product_id'])
del df_piq_table
data.drop(['product_id'], inplace = True, axis = 1)

data = reduce_mem_usage(data)
data.to_hdf('data_variables.h5', key = 'data_variables', mode = 'w')
display(data.head())
display(data.tail())

Mem. usage decreased to 13293.01 Mb (6.7% reduction)


,day,demand,demand_rolling_kurt_t30,demand_rolling_skew_t30,demand_rolling_std_t180,demand_rolling_std_t30,demand_rolling_std_t365,demand_rolling_std_t7,demand_rolling_std_t90,id,part,wm_yr_wk,SuperBowl,SuperBowl_near,SuperBowl_week,SuperBowl_weekend,ValentinesDay,ValentinesDay_near,ValentinesDay_week,ValentinesDay_weekend,PresidentsDay,PresidentsDay_near,PresidentsDay_week,PresidentsDay_weekend,LentStart,LentStart_near,LentStart_week,LentStart_weekend,LentWeek2,LentWeek2_near,LentWeek2_week,LentWeek2_weekend,StPatricksDay,StPatricksDay_near,StPatricksDay_week,StPatricksDay_weekend,Purim End,Purim End_near,Purim End_week,Purim End_weekend,OrthodoxEaster,OrthodoxEaster_near,OrthodoxEaster_week,OrthodoxEaster_weekend,Pesach End,Pesach End_near,...,price_change_mean_t13_bin_0.10-0.25,price_change_mean_t13_bin_0.25++,price_change_mean_t26_bin_--0.25,price_change_mean_t26_bin_-0.25-0.10,price_change_mean_t26_bin_-0.10-0.5,price_change_mean_t26_bin_-0.05-0,price_change_mean_t26_bin_0.05-0.10,price_change_mean_t26_bin_0.10-0.25,price_change_mean_t26_bin_0.25++,price_change_mean_t52_bin_--0.25,price_change_mean_t52_bin_-0.25-0.10,price_change_mean_t52_bin_-0.10-0.5,price_change_mean_t52_bin_-0.05-0,price_change_mean_t52_bin_0.05-0.10,price_change_mean_t52_bin_0.10-0.25,price_change_mean_t52_bin_0.25++,snap,snap_other,snap_only_other,season_1,season_2,season_4,month_1,month_2,month_3,month_4,month_6,month_7,month_8,month_9,month_10,month_11,month_12,wday_1,wday_2,wday_3,wday_4,wday_6,wday_7,month_fase_1,month_fase_3,week_fase_1,week_fase_3,p_parameter,i_parameter,q_parameter
0,d_1,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOBBIES_1_008_CA_1_validation,train,11101,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,10,1,1
1,d_1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOBBIES_1_009_CA_1_validation,train,11101,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,13,1,1
2,d_1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOBBIES_1_010_CA_1_validation,train,11101,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1
3,d_1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOBBIES_1_012_CA_1_validation,train,11101,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,2,0,2
4,d_1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HOBBIES_1_015_CA_1_validation,train,11101,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,1


,day,demand,demand_rolling_kurt_t30,demand_rolling_skew_t30,demand_rolling_std_t180,demand_rolling_std_t30,demand_rolling_std_t365,demand_rolling_std_t7,demand_rolling_std_t90,id,part,wm_yr_wk,SuperBowl,SuperBowl_near,SuperBowl_week,SuperBowl_weekend,ValentinesDay,ValentinesDay_near,ValentinesDay_week,ValentinesDay_weekend,PresidentsDay,PresidentsDay_near,PresidentsDay_week,PresidentsDay_weekend,LentStart,LentStart_near,LentStart_week,LentStart_weekend,LentWeek2,LentWeek2_near,LentWeek2_week,LentWeek2_weekend,StPatricksDay,StPatricksDay_near,StPatricksDay_week,StPatricksDay_weekend,Purim End,Purim End_near,Purim End_week,Purim End_weekend,OrthodoxEaster,OrthodoxEaster_near,OrthodoxEaster_week,OrthodoxEaster_weekend,Pesach End,Pesach End_near,...,price_change_mean_t13_bin_0.10-0.25,price_change_mean_t13_bin_0.25++,price_change_mean_t26_bin_--0.25,price_change_mean_t26_bin_-0.25-0.10,price_change_mean_t26_bin_-0.10-0.5,price_change_mean_t26_bin_-0.05-0,price_change_mean_t26_bin_0.05-0.10,price_change_mean_t26_bin_0.10-0.25,price_change_mean_t26_bin_0.25++,price_change_mean_t52_bin_--0.25,price_change_mean_t52_bin_-0.25-0.10,price_change_mean_t52_bin_-0.10-0.5,price_change_mean_t52_bin_-0.05-0,price_change_mean_t52_bin_0.05-0.10,price_change_mean_t52_bin_0.10-0.25,price_change_mean_t52_bin_0.25++,snap,snap_other,snap_only_other,season_1,season_2,season_4,month_1,month_2,month_3,month_4,month_6,month_7,month_8,month_9,month_10,month_11,month_12,wday_1,wday_2,wday_3,wday_4,wday_6,wday_7,month_fase_1,month_fase_3,week_fase_1,week_fase_3,p_parameter,i_parameter,q_parameter
47735392,d_1969,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOODS_3_823_WI_3_validation,test2,11621,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,8,1,1
47735393,d_1969,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOODS_3_824_WI_3_validation,test2,11621,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,10,1,1
47735394,d_1969,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOODS_3_825_WI_3_validation,test2,11621,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,8,1,1
47735395,d_1969,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOODS_3_826_WI_3_validation,test2,11621,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,9,1,1
47735396,d_1969,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOODS_3_827_WI_3_validation,test2,11621,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1


In [16]:
#Considering that we will model an ARIMA by product, we will already construct a database separed by product.
#The ideal would be to use the key function of hdf files, but this would exceed the recomented nunber of children (16.384).
#Therefore, we will make one file by product.

products_ids = data['id'].unique()
products_ids_size = len(products_ids)

progress = 0   #Usefull to see the progress of the code
progress_1000 = 1
start = time.time()

path = 'C:\\Users\\maxwi\\Python\\Kaggle\\M5 Forecasting - Accuracy\\Modelo 1\\variables\\data_by_id\\'
for e in products_ids:
    data.loc[np.in1d(data['id'], e)].to_hdf(path + e + '.h5', key = e, mode = 'w')
    
    progress += 1
    if progress > progress_1000 * 1000:

        progress_per = round(progress / products_ids_size, 4)
        print(progress_per)
        progress_1000 +=1
        
        end = time.time()
        elapsed = int(round(end - start, 0))
        total_run_time =  int(round(elapsed / (progress_per), 0))
        time_to_finish = int(round(elapsed / (progress_per), 0)) - elapsed
        print('Elapsed: {:02d}:{:02d}:{:02d}'.format(elapsed // 3600, (elapsed % 3600 // 60), elapsed % 60))
        print('Total run time: {:02d}:{:02d}:{:02d}'.format(total_run_time // 3600, (total_run_time % 3600 // 60), total_run_time % 60))
        print('Time to finish: {:02d}:{:02d}:{:02d}'.format(time_to_finish // 3600, (time_to_finish % 3600 // 60), time_to_finish % 60))
        print()

0.0328
Elapsed: 00:10:34
Total run time: 05:22:09
Time to finish: 05:11:35

0.0656
Elapsed: 00:21:03
Total run time: 05:20:53
Time to finish: 04:59:50

0.0984
Elapsed: 00:31:32
Total run time: 05:20:28
Time to finish: 04:48:56

0.1312
Elapsed: 00:42:09
Total run time: 05:21:16
Time to finish: 04:39:07

0.164
Elapsed: 00:52:41
Total run time: 05:21:14
Time to finish: 04:28:33

0.1968
Elapsed: 01:03:16
Total run time: 05:21:29
Time to finish: 04:18:13

0.2296
Elapsed: 01:13:50
Total run time: 05:21:34
Time to finish: 04:07:44

0.2624
Elapsed: 01:24:29
Total run time: 05:21:58
Time to finish: 03:57:29

0.2952
Elapsed: 01:34:59
Total run time: 05:21:46
Time to finish: 03:46:47

0.328
Elapsed: 01:45:28
Total run time: 05:21:33
Time to finish: 03:36:05

0.3608
Elapsed: 01:56:16
Total run time: 05:22:15
Time to finish: 03:25:59

0.3936
Elapsed: 02:06:59
Total run time: 05:22:37
Time to finish: 03:15:38

0.4264
Elapsed: 02:17:30
Total run time: 05:22:28
Time to finish: 03:04:58

0.4592
Elapsed